# ProtoCLR

https://huggingface.co/ilyassmoummad/ProtoCLR

In [1]:
!git clone https://huggingface.co/ilyassmoummad/ProtoCLR

Cloning into 'ProtoCLR'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 93 (delta 45), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (93/93), 41.14 KiB | 638.00 KiB/s, done.
Filtering content: 100% (4/4), 315.15 MiB | 49.22 MiB/s, done.


In [2]:
!cd ProtoCLR/

In [8]:
!pip install -r requirements.txt

In [12]:
!pip install pydub

In [14]:
from pydub import AudioSegment

# Input and output file paths
input_file = "/content/XC267683.ogg"
output_file = "/content/XC267683.wav"

# Load the OGG file
audio = AudioSegment.from_file(input_file, format="ogg")

# Export as WAV
audio.export(output_file, format="wav")
print(f"Converted {input_file} to {output_file}")

Converted /content/XC267683.ogg to /content/XC267683.wav


In [19]:
import torch
from cvt import cvt13  # Import model architecture
from melspectrogram import MelSpectrogramProcessor  # Import Mel spectrogram processor
import torchaudio
from torchaudio.transforms import Resample

# Initialize the preprocessor and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
preprocessor = MelSpectrogramProcessor(device=device)
model = cvt13().to(device)

# Load weights trained using Cross-Entropy
model.load_state_dict(torch.load("ce.pth", map_location="cpu")['encoder'])

# Load weights trained using SimCLR (self-supervised contrastive learning)
model.load_state_dict(torch.load("simclr.pth", map_location="cpu"))

# Load weights trained using SupCon (supervised contrastive learning)
model.load_state_dict(torch.load("supcon.pth", map_location="cpu"))

# Load weights trained using ProtoCLR (supervised contrastive learning using prototypes)
model.load_state_dict(torch.load("protoclr.pth", map_location="cpu"))

# Optional: Move the model to GPU for faster processing if available using : model = model.to('cuda') , for instance.
model.eval()

# Load and preprocess a sample audio waveform
def load_waveform(file_path, target_sample_rate=16000):
    # Replace this with your specific audio loading function
    # For example, using torchaudio to load and resample
    # Load the audio file
    waveform, sample_rate = torchaudio.load(file_path)  # Shape: [channel, samples]

    # If the audio has multiple channels, convert to mono by averaging
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Resample if the sample rate is not the target sample rate
    if sample_rate != target_sample_rate:
        resampler = Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
        waveform = resampler(waveform)

    return waveform.squeeze(0)  # Return 1D waveform
    # pass

waveform = load_waveform("/content/XC267683.wav").to(device)  # Load your audio file here and convert it to a PyTorch tensor.

# Ensure waveform is sampled at 16 kHz, then pad/chunk as needed for 6s length
input_tensor = preprocessor.process(waveform).unsqueeze(0)  # Add batch dimension

# Run the model on the preprocessed audio
with torch.no_grad():
    output = model(input_tensor)
    print("Model output shape:", output.shape)

ValueError: not enough values to unpack (expected 4, got 3)